In [ ]:
# Download a file based on its file ID.
file_id = '1HADYvII1moEln8JwB_fOEb1XqlElNcK3'
!gdown https://drive.google.com/uc?id={file_id}

# Unzip 
!unzip -q Plant_Disease_DataX.zip

Downloading...
From: https://drive.google.com/uc?id=1HADYvII1moEln8JwB_fOEb1XqlElNcK3
To: /content/Plant_Disease_DataX.zip
100% 866M/866M [00:06<00:00, 130MB/s] 


In [ ]:
import os
root_dir = './PlantVillage'
train_dir = os.path.join(root_dir, 'train')
val_dir = os.path.join(root_dir, 'val')

In [ ]:
from os import listdir
plant_diseases = listdir(train_dir)
plant_diseases

['Soybean___healthy',
 'Grape___healthy',
 'Apple___Cedar_apple_rust',
 'Tomato___Septoria_leaf_spot',
 'Apple___Apple_scab',
 'Blueberry___healthy',
 'Raspberry___healthy',
 'Potato___healthy',
 'Tomato___Late_blight',
 'Orange___Haunglongbing_(Citrus_greening)',
 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
 'Apple___healthy',
 'background',
 'Pepper,_bell___Bacterial_spot',
 'Potato___Early_blight',
 'Strawberry___Leaf_scorch',
 'Cherry_(including_sour)___healthy',
 'Strawberry___healthy',
 'Tomato___Spider_mites Two-spotted_spider_mite',
 'Corn_(maize)___Common_rust_',
 'Peach___healthy',
 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 'Potato___Late_blight',
 'Peach___Bacterial_spot',
 'Cherry_(including_sour)___Powdery_mildew',
 'Tomato___Leaf_Mold',
 'Tomato___Bacterial_spot',
 'Tomato___healthy',
 'Squash___Powdery_mildew',
 'Pepper,_bell___healthy',
 'Corn_(maize)___healthy',
 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 'Tomato___Early_blight',
 'Apple___Black_rot',
 '

In [ ]:
import cv2
import numpy as np
from keras.preprocessing.image import img_to_array
def image_to_array(dir):
  image = cv2.imread(dir)
  image = cv2.resize(image, tuple((256,256)))
  return img_to_array(image)
 

In [ ]:
label = []
image_list = []
for plant_disease in plant_diseases:
  plant_image = listdir(f"{train_dir}/{plant_disease}/")
  for image in plant_image[:100]:
    image_dir = f"{train_dir}/{plant_disease}/{image}"
    image_list.append(image_to_array(image_dir))
    label.append(plant_disease)


In [ ]:
array_image_list = np.array(image_list, dtype=np.float16)/ 225.0

In [ ]:
from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
image_labels = label_binarizer.fit_transform(label)
classes = len(label_binarizer.classes_)
print(classes)

39


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(
    rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, 
    zoom_range=0.2,horizontal_flip=True, 
    fill_mode="nearest")

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(array_image_list, image_labels, test_size=0.2, random_state = 42)

In [ ]:
from tensorflow.keras.models import Sequential
from keras.layers.normalization import layer_normalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.core import Activation, Flatten, Dropout, Dense
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
import tensorflow as tf

In [ ]:
EPOCHS = 25
STEPS = 100
LR = 1e-3
BATCH_SIZE = 32
WIDTH = 256
HEIGHT = 256
DEPTH = 3

In [ ]:
x_train[0].shape

(256, 256, 3)

In [ ]:

model = Sequential()
inputShape = (HEIGHT, WIDTH, DEPTH)

model.add(Conv2D(32, (3, 3), padding="same",input_shape=inputShape))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(3, 3)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(64, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(Conv2D(128, (3, 3), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Activation("softmax"))

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 32)      896       
_________________________________________________________________
activation (Activation)      (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 85, 85, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 85, 85, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 85, 85, 64)        18496     
_________________________________________________________________
activation_1 (Activation)    (None, 85, 85, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 85, 85, 64)        3

In [ ]:
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
x_train.shape

(3120, 256, 256, 3)

In [ ]:
y_test[:5]

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [ ]:
model.fit(x_train,y_train, epochs = 5)

Epoch 1/5


In [ ]:
y_train.shape

(3120, 39)

In [ ]:
y_train

array([[0, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])